In [18]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

In [19]:
column_names = list(range(0,7))
column_names.append('Classification')

df = pd.read_csv("pima-indians-diabetes.data", names = column_names) 
df

,0,1,2,3,4,5,6,Classification
6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.167,21,0
0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...
10,101,76,48,180,32.9,0.171,63,0
2,122,70,27,0,36.8,0.340,27,0
5,121,72,23,112,26.2,0.245,30,0
1,126,60,0,0,30.1,0.349,47,1


In [20]:
df.dtypes

0                   int64
1                   int64
2                   int64
3                   int64
4                 float64
5                 float64
6                   int64
Classification      int64
dtype: object

In [21]:
#find number of null entries to see if there is a need to clean the dataset
df.isnull().sum()
#df = df.dropna()

0                 0
1                 0
2                 0
3                 0
4                 0
5                 0
6                 0
Classification    0
dtype: int64

In [22]:
df.nunique()

0                 136
1                  47
2                  51
3                 186
4                 248
5                 517
6                  52
Classification      2
dtype: int64

In [23]:
X = df.drop('Classification', axis=1)
y = df['Classification']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [24]:
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

tf.random.set_seed(42)
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(128, activation='relu', input_shape = (7,)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy",
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

from sklearn.model_selection import KFold
kf = KFold(n_splits=10, random_state =42, shuffle=True)

sum_acc_score = 0
sum_prec_score = 0
sum_recall_score = 0
sum_f1_score = 0


for train_index, test_index in kf.split(X):
    X_train = pd.DataFrame()
    y_train = pd.DataFrame()

    X_test = pd.DataFrame()
    y_test = pd.DataFrame()
    
    for index in train_index:
        X_train = X_train.append(X.iloc[index])
        y_train = y_train.append(pd.Series(y.iloc[index]), ignore_index = True)
        
    for index in test_index:
        X_test = X_test.append(X.iloc[index])
        y_test = y_test.append(pd.Series(y.iloc[index]), ignore_index = True)

    train_X = tf.convert_to_tensor(X_train)
    test_X = tf.convert_to_tensor(X_test)

    #one hot encode y_train and y_test
    train_y = tf.keras.utils.to_categorical(y_train)
    test_y = tf.keras.utils.to_categorical(y_test)

            
    history = model.fit(train_X, train_y, epochs=50, validation_data = (test_X, test_y),callbacks=[callback])
    predictions = model.predict(test_X)
    
    predicted_classes = predictions.argmax(axis=1)
    test_y_classes = test_y.argmax(axis=1)
    
    sum_acc_score += accuracy_score(test_y_classes, predicted_classes)
    sum_prec_score += precision_score(test_y_classes, predicted_classes, average = 'micro')
    sum_recall_score += recall_score(test_y_classes, predicted_classes, average = 'micro')
    sum_f1_score += f1_score(test_y_classes, predicted_classes, average = 'micro')

Epoch 1/50
22/22 [==============================] - 1s 23ms/step - loss: 1.2967 - accuracy: 0.5948 - precision: 0.5948 - recall: 0.5948 - val_loss: 0.8094 - val_accuracy: 0.6883 - val_precision: 0.6883 - val_recall: 0.6883
Epoch 2/50
22/22 [==============================] - 0s 4ms/step - loss: 1.0537 - accuracy: 0.6295 - precision: 0.6295 - recall: 0.6295 - val_loss: 0.7849 - val_accuracy: 0.7273 - val_precision: 0.7273 - val_recall: 0.7273
Epoch 3/50
22/22 [==============================] - 0s 3ms/step - loss: 0.7440 - accuracy: 0.6469 - precision: 0.6469 - recall: 0.6469 - val_loss: 0.8766 - val_accuracy: 0.6104 - val_precision: 0.6104 - val_recall: 0.6104
Epoch 4/50
22/22 [==============================] - 0s 3ms/step - loss: 0.6819 - accuracy: 0.6889 - precision: 0.6889 - recall: 0.6889 - val_loss: 0.7222 - val_accuracy: 0.6623 - val_precision: 0.6623 - val_recall: 0.6623
Epoch 5/50
22/22 [==============================] - 0s 4ms/step - loss: 0.6212 - accuracy: 0.7033 - precision: 

22/22 [==============================] - 0s 4ms/step - loss: 0.7181 - accuracy: 0.6990 - precision: 0.6990 - recall: 0.6990 - val_loss: 1.0734 - val_accuracy: 0.5974 - val_precision: 0.5974 - val_recall: 0.5974
Epoch 5/50
22/22 [==============================] - 0s 4ms/step - loss: 0.6298 - accuracy: 0.7192 - precision: 0.7192 - recall: 0.7192 - val_loss: 0.9838 - val_accuracy: 0.6494 - val_precision: 0.6494 - val_recall: 0.6494
Epoch 6/50
22/22 [==============================] - 0s 4ms/step - loss: 0.5898 - accuracy: 0.7366 - precision: 0.7366 - recall: 0.7366 - val_loss: 0.7042 - val_accuracy: 0.6364 - val_precision: 0.6364 - val_recall: 0.6364
Epoch 7/50
22/22 [==============================] - 0s 4ms/step - loss: 0.5626 - accuracy: 0.7279 - precision: 0.7279 - recall: 0.7279 - val_loss: 0.6553 - val_accuracy: 0.7013 - val_precision: 0.7013 - val_recall: 0.7013
Epoch 8/50
22/22 [==============================] - 0s 3ms/step - loss: 0.5693 - accuracy: 0.7308 - precision: 0.7308 - rec

22/22 [==============================] - 0s 5ms/step - loss: 0.5079 - accuracy: 0.7467 - precision: 0.7467 - recall: 0.7467 - val_loss: 0.5541 - val_accuracy: 0.7532 - val_precision: 0.7532 - val_recall: 0.7532
Epoch 14/50
22/22 [==============================] - 0s 5ms/step - loss: 0.5105 - accuracy: 0.7453 - precision: 0.7453 - recall: 0.7453 - val_loss: 0.5073 - val_accuracy: 0.7662 - val_precision: 0.7662 - val_recall: 0.7662
Epoch 15/50
22/22 [==============================] - 0s 4ms/step - loss: 0.5118 - accuracy: 0.7424 - precision: 0.7424 - recall: 0.7424 - val_loss: 0.5065 - val_accuracy: 0.7532 - val_precision: 0.7532 - val_recall: 0.7532
Epoch 16/50
22/22 [==============================] - 0s 4ms/step - loss: 0.5121 - accuracy: 0.7381 - precision: 0.7381 - recall: 0.7381 - val_loss: 0.5179 - val_accuracy: 0.7532 - val_precision: 0.7532 - val_recall: 0.7532
Epoch 17/50
22/22 [==============================] - 0s 4ms/step - loss: 0.5043 - accuracy: 0.7569 - precision: 0.7569 -

22/22 [==============================] - 0s 4ms/step - loss: 0.4581 - accuracy: 0.8017 - precision: 0.8017 - recall: 0.8017 - val_loss: 0.6334 - val_accuracy: 0.7013 - val_precision: 0.7013 - val_recall: 0.7013
Epoch 10/50
22/22 [==============================] - 0s 4ms/step - loss: 0.4384 - accuracy: 0.8032 - precision: 0.8032 - recall: 0.8032 - val_loss: 0.4985 - val_accuracy: 0.7143 - val_precision: 0.7143 - val_recall: 0.7143
Epoch 11/50
22/22 [==============================] - 0s 4ms/step - loss: 0.4205 - accuracy: 0.8032 - precision: 0.8032 - recall: 0.8032 - val_loss: 0.5529 - val_accuracy: 0.6753 - val_precision: 0.6753 - val_recall: 0.6753
Epoch 12/50
22/22 [==============================] - 0s 4ms/step - loss: 0.4493 - accuracy: 0.8003 - precision: 0.8003 - recall: 0.8003 - val_loss: 0.5276 - val_accuracy: 0.6883 - val_precision: 0.6883 - val_recall: 0.6883
Epoch 13/50
3/3 [==============================] - 0s 2ms/step
Epoch 1/50
22/22 [==============================] - 0s 5m

22/22 [==============================] - 0s 5ms/step - loss: 0.4248 - accuracy: 0.7916 - precision: 0.7916 - recall: 0.7916 - val_loss: 0.4674 - val_accuracy: 0.7532 - val_precision: 0.7532 - val_recall: 0.7532
Epoch 3/50
22/22 [==============================] - 0s 6ms/step - loss: 0.3882 - accuracy: 0.8292 - precision: 0.8292 - recall: 0.8292 - val_loss: 0.4572 - val_accuracy: 0.7792 - val_precision: 0.7792 - val_recall: 0.7792
Epoch 4/50
22/22 [==============================] - 0s 6ms/step - loss: 0.3930 - accuracy: 0.8177 - precision: 0.8177 - recall: 0.8177 - val_loss: 0.4843 - val_accuracy: 0.7792 - val_precision: 0.7792 - val_recall: 0.7792
Epoch 5/50
22/22 [==============================] - 0s 5ms/step - loss: 0.3878 - accuracy: 0.8191 - precision: 0.8191 - recall: 0.8191 - val_loss: 0.4914 - val_accuracy: 0.7532 - val_precision: 0.7532 - val_recall: 0.7532
Epoch 6/50
22/22 [==============================] - 0s 6ms/step - loss: 0.4132 - accuracy: 0.8177 - precision: 0.8177 - rec

In [25]:
print('Average Accuracy Score: ', sum_acc_score/10)
print('Average Precision Score: ', sum_prec_score/10)
print('Average Recall Score: ', sum_recall_score/10)
print('Average F1-Score: ', sum_f1_score/10)



Average Accuracy Score:  0.748855092276145
Average Precision Score:  0.748855092276145
Average Recall Score:  0.748855092276145
Average F1-Score:  0.748855092276145
